<a href="https://colab.research.google.com/github/xuweicheng/ml-poc/blob/main/miles_per_gallon_poc.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
automobile_df = pd.read_csv('auto-mpg.csv')

automobile_df.head()

,mpg,cylinders,displacement,horsepower,weight,acceleration,model year,origin,car name
0,18.0,8,307.0,130,3504,12.0,70,1,chevrolet chevelle malibu
1,15.0,8,350.0,165,3693,11.5,70,1,buick skylark 320
2,18.0,8,318.0,150,3436,11.0,70,1,plymouth satellite
3,16.0,8,304.0,150,3433,12.0,70,1,amc rebel sst
4,17.0,8,302.0,140,3449,10.5,70,1,ford torino


In [3]:
automobile_df.shape

(398, 9)

In [4]:
automobile_df.dtypes

mpg             float64
cylinders         int64
displacement    float64
horsepower       object
weight            int64
acceleration    float64
model year        int64
origin            int64
car name         object
dtype: object

In [5]:
automobile_df["horsepower"] = pd.to_numeric(automobile_df['horsepower'], errors='coerce')

In [8]:
automobile_df.isnull().sum()

mpg             0
cylinders       0
displacement    0
horsepower      0
weight          0
acceleration    0
model year      0
origin          0
car name        0
dtype: int64

In [7]:
automobile_df = automobile_df.fillna(automobile_df.mean())

<ipython-input-7-2b3e8e8759ec>:1: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  automobile_df = automobile_df.fillna(automobile_df.mean())


In [9]:
automobile_df.isna().sum()

mpg             0
cylinders       0
displacement    0
horsepower      0
weight          0
acceleration    0
model year      0
origin          0
car name        0
dtype: int64

In [10]:
automobile_df = automobile_df.drop(['car name'], axis=1)

In [11]:
automobile_df.head()

,mpg,cylinders,displacement,horsepower,weight,acceleration,model year,origin
0,18.0,8,307.0,130.0,3504,12.0,70,1
1,15.0,8,350.0,165.0,3693,11.5,70,1
2,18.0,8,318.0,150.0,3436,11.0,70,1
3,16.0,8,304.0,150.0,3433,12.0,70,1
4,17.0,8,302.0,140.0,3449,10.5,70,1


In [13]:
X = automobile_df.drop('mpg', axis = 1)
Y = automobile_df['mpg']

In [14]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

In [16]:
from sklearn import linear_model
from sklearn.pipeline import make_pipeline

x_train, x_test, y_train, y_test=train_test_split(X, Y, test_size=0.2)

linear_model = LinearRegression(normalize=True).fit(x_train, y_train)

print('R2: ', linear_model.score(x_train, y_train))
y_pred = linear_model.predict(x_test)
print('Mean Squared Error: ', mean_squared_error(y_test, y_pred))


R2:  0.8214388343637367
Mean Squared Error:  13.519315986001029


/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_base.py:141: FutureWarning: 'normalize' was deprecated in version 1.0 and will be removed in 1.2.
If you wish to scale the data, use Pipeline with a StandardScaler in a preprocessing stage. To reproduce the previous behavior:

from sklearn.pipeline import make_pipeline

model = make_pipeline(StandardScaler(with_mean=False), LinearRegression())

If you wish to pass a sample_weight parameter, you need to pass it as a fit parameter to each step of the pipeline as follows:

kwargs = {s[0] + '__sample_weight': sample_weight for s in model.steps}
model.fit(X, y, **kwargs)


  warnings.warn(


In [19]:
print(linear_model.predict([[6, 117, 100, 3000, 15, 85, 1]]))

[28.03750063]


/usr/local/lib/python3.8/dist-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(


In [20]:
import pickle

In [21]:
pickle.dump(linear_model, open('model.pkl', 'wb'))

In [22]:
model = pickle.load(open('model.pkl', 'rb'))

In [23]:
!pip install flask-ngrok

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [26]:
from flask_ngrok import run_with_ngrok
from flask import Flask, request
import requests
import numpy as np
app = Flask(__name__)
run_with_ngrok(app)
@app.route('/predict', methods=['POST'])
def home():
  cylinders = int(request.args.get('cylinders', ''))
  displacement = int(request.args.get('displacement', ''))
  horsepower = int(request.args.get('horsepower', ''))
  weight = int(request.args.get('weight', ''))
  acceleration = int(request.args.get('acceleration', ''))
  modelYear = int(request.args.get('modelYear', ''))
  origin = int(request.args.get('origin', ''))
  prediction = model.predict([[cylinders, displacement, horsepower, weight, acceleration, modelYear, origin]])
  print('*******')
  print(prediction)
  return 'Prediction is ' + str(prediction[0])

app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


INFO:werkzeug: * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://6cf0-34-125-142-237.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


/usr/local/lib/python3.8/dist-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(
INFO:werkzeug:127.0.0.1 - - [20/Feb/2023 01:37:52] "POST /predict?cylinders=6&displacement=117&horsepower=100&weight=3000&acceleration=15&modelYear=85&origin=1 HTTP/1.1" 200 -


*******
[28.03750063]
